In [1]:
from evaluations import *

Loading pyMOR version 0.4.2-116-gadb475af


In [2]:
domain = RectDomain(
    domain=([0, 0], [1, 1]),
    left=BoundaryType('robin'),
    right=BoundaryType('robin'),
    top=BoundaryType('dirichlet'),
    bottom=BoundaryType('dirichlet'))

excitation = GenericFunction(
    lambda x: (
        0j + np.outer(np.exp(-np.linalg.norm(x - np.array([0.1,0.5]), axis=1)**2 *200) , np.array([0., 1.]))
        )
        ,
    dim_domain=2,
    shape_range=(2,)
)

#excitation = ExpressionFunction('outer((linalg.norm(x - array([0.2,0.5]), axis=1)**2 <= 0.001) * 1., array([0., 1.]))', dim_domain=2,shape_range=(2,))

robinvalue = -1

def func(x, mu):
    return ((x[...,0]>1-1e-13)+(x[...,0]<1e-13)+(x[...,1]>1-1e-13)+(x[...,1]<1e-13)>0)*robinvalue +((x[...,0]>1-1e-13)+(x[...,0]<1e-13)+(x[...,1]>1-1e-13)+(x[...,1]<1e-13)==0)* mu["c_loc"]
#return ((x[...,0]==1)+(x[...,0]==0)+(x[...,1]==1)+(x[...,1]==0)>0)*robinvalue +((x[...,0]==1)+(x[...,0]==0)+(x[...,1]==1)+(x[...,1]==0)==0)* mu["c_loc"]
robin_data = GenericFunction(func, dim_domain =2, parameter_type={'c_loc' : (), 'k': (), 'mu': (), 'eps': ()})
#robin_data = ConstantFunction(robinvalue, dim_domain = 2)
parameter_range=(0., 100.)
p = MaxwellProblem(domain=domain, robin_data=robin_data, excitation=excitation,
     curl_functionals = [ExpressionParameterFunctional('1./mu', parameter_type={'c_loc' : (), 'k': (), 'mu': (), 'eps': ()}, )], 
    reaction_functionals = [ExpressionParameterFunctional('-k**2*eps', parameter_type={'c_loc' : (), 'k': (), 'mu': (), 'eps': ()})], 
    robin_functionals = [ExpressionParameterFunctional('1j*k', parameter_type={'c_loc' : (), 'k': (), 'mu': (), 'eps': ()})],
    rhs_functionals = [ExpressionParameterFunctional('-1*k', parameter_type={'c_loc' : (), 'k': (), 'mu': (), 'eps': ()})]
)
diameter = 1./100
d, data = discretize_maxwell(p, diameter = diameter)
mus = {'c_loc' : 0, 'k': 10, 'mu': 1, 'eps': 1}
u = d.solve(mus)
d.visualize(u)

/home/root_dennis/pymor_MA/src/pymor/gui/qt.py:282: ComplexWarning: Casting complex values to real discards the imaginary part
  tuple(u.data.astype(np.float64, copy=False) for u in U)


In [3]:
d.products

{'h1': <pymor.operators.constructions.LincombOperator at 0x7f48512ca790>,
 'h1_0': <pymor.operators.constructions.LincombOperator at 0x7f48512cab10>,
 'h1_semi': <pymor.operators.nedelec.RotRotOperator at 0x7f486665cf10>,
 'l2': <pymor.operators.nedelec.L2ProductOperator at 0x7f48512ca590>,
 'l2_0': <pymor.operators.nedelec.L2ProductOperator at 0x7f48512ca810>}

In [18]:
#p = maxwell_problem()
resolution = 100
coarse_grid_resolution = 10

mus = {'c_loc' : 10j, 'k': 10, 'mu': 1, 'eps': 1}
#mus = {'c_loc' : 0., 'k': 1e9*2*np.pi, 'mu': 4*np.pi*1e-7, 'eps': 8.854187817e-12}
gq, lq = localize_problem(p, coarse_grid_resolution, resolution, mus, dof_codim = 1, discretizer = discretize_maxwell)

localizing problem


In [19]:
basis = create_bases2(gq, lq, 30, transfer = 'robin', silent = False)
ru = reconstruct_solution(gq,lq,basis, silent = False)

creating bases
reconstructing solution


In [20]:
d = gq["d"]
u = d.solve(mus)
dif = u-ru

In [21]:
dif = u-ru
norm = gq["full_norm"]
print norm(dif)[0]/norm(u)[0]
d.visualize((dif.real, dif.imag, u.real, u.imag, ru.real, ru.imag), legend = ('dif.real', 'dif.imag', 'u.real', 'u.imag', 'ru.real', 'ru.imag'), separate_colorbars = True)

6.91788655645e-06


/home/root_dennis/pymor_MA/src/pymor/gui/qt.py:282: ComplexWarning: Casting complex values to real discards the imaginary part
  tuple(u.data.astype(np.float64, copy=False) for u in U)


7.29851552621e-05
